# 题目

> Alice 和 Bob 共有一个无向图，其中包含 n 个节点和 3  种类型的边：  
类型 1：只能由 Alice 遍历。  
类型 2：只能由 Bob 遍历。  
类型 3：Alice 和 Bob 都可以遍历。  
给你一个数组 edges ，其中 edges[i] = [typei, ui, vi] 表示节点 ui 和 vi 之间存在类型为 typei 的双向边。请你在保证图仍能够被 Alice和 Bob 完全遍历的前提下，找出可以删除的最大边数。如果从任何节点开始，Alice 和 Bob 都可以到达所有其他节点，则认为图是可以完全遍历的。  
返回可以删除的最大边数，如果 Alice 和 Bob 无法完全遍历图，则返回 -1 。

# 方法一：并查集

> 类型 1,2,3 分别为「Alice 独占边」「Bob 独占边」以及「公共边」。  
对于每个人来说，都需要仅通过独占边和公共边将所有节点连接到一个集合中。删除最多的边相当于保留最少的边，因此可以先将所有边去掉，只剩下节点，然后在将边添加进入。  
因为公共边对两个人都有效，因此严格优于独占边，先添加公共边，在分别添加两者的独占边。  
最终，若两者的图中都被连通，则返回答案，否则返回-1。

## 复杂度

- 时间复杂度: $O(m\alpha(n))$ ，其中 $m$ 是数组 edges 的长度， $n$ 是节点的个数， $\alpha$ 是阿克曼函数的反函数。

- 空间复杂度: $O(n)$ ，其中 $m$ 是行数， $n$ 是节点的个数。

> 并查集所需的空间。

## 代码

In [1]:
# 并查集模板
class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))  # 记录节点及其指向情况
        self.size = [1] * n  # 记录代表元素对应的集合的大小（节点数量）
        self.n = n
        # 当前集合的数目（共有n个节点，彼此不连通，所以有n个集合）
        self.setCount = n
    
    # 寻找某节点所属的集合的代表元素，并进行路径压缩
    def findset(self, x):
        if self.parent[x] == x:
            return x
        self.parent[x] = self.findset(self.parent[x])
        return self.parent[x]
    
    # 合并两个节点所属的集合
    def unite(self, x, y):
        x, y = self.findset(x), self.findset(y)
        if x == y:
            return False
        # 按size合并
        if self.size[x] < self.size[y]:
            x, y = y, x
        self.parent[y] = x
        self.size[x] += self.size[y]  # 更新size
        self.setCount -= 1  # 每次合并成功，总集合数量-1
        return True
    
    # 判断两个节点是否属于同一集合
    def connected(self, x, y):
        x, y = self.findset(x), self.findset(y)
        return x == y

# 解法
class Solution:
    def maxNumEdgesToRemove(self, n, edges):
        
        ufa, ufb = UnionFind(n), UnionFind(n)  # 分别给Alice和Bob各构建一个并查集
        
        ans = 0  # 可删减的边数
        
        # 节点编号改为从 0 开始，因此把edges列表中表示节点的数都-1
        for edge in edges:
            edge[1] -= 1
            edge[2] -= 1

        # 先使用公共边连接节点
        for t, u, v in edges:
            if t == 3:
                # 若当前公共边对应的两个节点属于同一个集合，则说明这条边是多余的，可以删减
                # 在判断过程中，若ufa.unite(u, v)=True，则说明公共边不多余，此时ufa和ufb都进行了合并操作（因为ufa的操作在判断时已经完成）
                if not ufa.unite(u, v):
                    ans += 1
                # 否则，将两个节点连通（此时集合数-1）
                else:
                    ufb.unite(u, v)

        # 此时的ufa和ufb是相等的，都添加完了所有的不多余公共边
        # 再使用独占边连接节点
        for t, u, v in edges:
            # Alice 独占边
            if t == 1:
                # 判断边是否多余
                if not ufa.unite(u, v):
                    ans += 1
            # Bob 独占边
            elif t == 2:
                # 判断边是否多余
                if not ufb.unite(u, v):
                    ans += 1
        
        # 任何一个图中的集合数多余一都说明不能遍历
        if ufa.setCount != 1 or ufb.setCount != 1:
            return -1
        
        return ans

#### 测试一

In [2]:
n = 4
edges = [[3,1,2],[3,2,3],[1,1,3],[1,2,4],[1,1,2],[2,3,4]]

test = Solution()
test.maxNumEdgesToRemove(n, edges)

2

#### 测试二

In [3]:
n = 4
edges = [[3,1,2],[3,2,3],[1,1,4],[2,1,4]]

test = Solution()
test.maxNumEdgesToRemove(n, edges)

0

#### 测试三

In [4]:
n = 4
edges = [[3,2,3],[1,1,2],[2,3,4]]

test = Solution()
test.maxNumEdgesToRemove(n, edges)

-1